# Load Dataset

In [1]:
!pip install -q datasets
!pip install dahuffman
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.4 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn
from datasets import load_dataset
from transformers import ViTModel
import numpy as np
import torch
import cv2
import torch.nn as nn
from transformers import ViTModel, ViTConfig
from torchvision import transforms
from torch.optim import Adam
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch
import torch.nn as nn
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch
import numpy as np
from collections import Counter
import torch
from collections import Counter
from dahuffman import HuffmanCodec



In [3]:
dataset = load_dataset("Matthijs/snacks")
print(dataset)
print(dataset["train"].features['label'].names)

labels = dataset["train"].features["label"].names
num_labels = len(dataset["train"].features["label"].names)
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label
print(label2id)
print(id2label)

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 4838
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 952
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 955
    })
})
['apple', 'banana', 'cake', 'candy', 'carrot', 'cookie', 'doughnut', 'grape', 'hot dog', 'ice cream', 'juice', 'muffin', 'orange', 'pineapple', 'popcorn', 'pretzel', 'salad', 'strawberry', 'waffle', 'watermelon']
{'apple': 0, 'banana': 1, 'cake': 2, 'candy': 3, 'carrot': 4, 'cookie': 5, 'doughnut': 6, 'grape': 7, 'hot dog': 8, 'ice cream': 9, 'juice': 10, 'muffin': 11, 'orange': 12, 'pineapple': 13, 'popcorn': 14, 'pretzel': 15, 'salad': 16, 'strawberry': 17, 'waffle': 18, 'watermelon': 19}
{0: 'apple', 1: 'banana', 2: 'cake', 3: 'candy', 4: 'carrot', 5: 'cookie', 6: 'doughnut', 7: 'grape', 8: 'hot dog', 9: 'ice cream', 10: 'juice', 11: 'muffin', 12: 'orange', 13: 'pineapple', 14: 'popcorn', 15: 'pretzel'

# Forward Pass ViT Model with a Toy Image

In [4]:
# Create toy image with dim (batch x channel x width x height)
toy_img = torch.rand(1, 3, 48, 48)

# Define conv layer parameters
num_channels = 3
hidden_size = 768
patch_size = 16

projection = nn.Conv2d(num_channels, hidden_size, kernel_size=patch_size,
             stride=patch_size)

out_projection = projection(toy_img)

print(f'Original image size: {toy_img.size()}')
print(f'Size after projection: {out_projection.size()}')


patch_embeddings = out_projection.flatten(2).transpose(1, 2)
print(f'Patch embedding size: {patch_embeddings.size()}')

# Define [CLS] token embedding with the same emb dimension as the patches
batch_size = 1
cls_token = nn.Parameter(torch.randn(1, 1, hidden_size))
cls_tokens = cls_token.expand(batch_size, -1, -1)

# Prepend [CLS] token in the beginning of patch embedding
patch_embeddings = torch.cat((cls_tokens, patch_embeddings), dim=1)
print(f'Patch embedding size: {patch_embeddings.size()}')

# Define position embedding with the same dimension as the patch embedding
position_embeddings = nn.Parameter(torch.randn(batch_size, 10, hidden_size))

# Add position embedding into patch embedding
input_embeddings = patch_embeddings + position_embeddings
print(f'Input embedding size: {input_embeddings.size()}')


# Define parameters for ViT-base (example)
num_heads = 12
num_layers = 12

# Define Transformer encoders' stack
transformer_encoder_layer = nn.TransformerEncoderLayer(
           d_model=hidden_size, nhead=num_heads,
           dim_feedforward=int(hidden_size * 4),
           dropout=0.1)
transformer_encoder = nn.TransformerEncoder(
           encoder_layer=transformer_encoder_layer,
           num_layers=num_layers)

# Forward pass
output_embeddings = transformer_encoder(input_embeddings)
print(f' Output embedding size: {output_embeddings.size()}')

# Load pretrained model
model_checkpoint = 'google/vit-base-patch16-224-in21k'
model = ViTModel.from_pretrained(model_checkpoint, add_pooling_layer=False)

# Example input image
input_img = torch.rand(batch_size, num_channels, 224, 224)

# Forward pass input image
output_embedding = model(input_img)
print(output_embedding)
print(f"Ouput embedding size: {output_embedding['last_hidden_state'].size()}")

num_labels = 20

# Define linear classifier layer
classifier = nn.Linear(hidden_size, num_labels)

# Forward pass on the output embedding of [CLS] token
output_classification = classifier(output_embedding['last_hidden_state'][:, 0, :])
print(f"Output embedding size: {output_classification.size()}")

Original image size: torch.Size([1, 3, 48, 48])
Size after projection: torch.Size([1, 768, 3, 3])
Patch embedding size: torch.Size([1, 9, 768])
Patch embedding size: torch.Size([1, 10, 768])
Input embedding size: torch.Size([1, 10, 768])
 Output embedding size: torch.Size([1, 10, 768])


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTModel: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing ViTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BaseModelOutputWithPooling(last_hidden_state=tensor([[[-0.0239, -0.2421,  0.0737,  ...,  0.2399,  0.0595, -0.3175],
         [ 0.0335, -0.1080, -0.0525,  ..., -0.0049,  0.1644, -0.2936],
         [ 0.1821, -0.2163, -0.0546,  ..., -0.0502,  0.0542, -0.3032],
         ...,
         [ 0.2061, -0.1157, -0.0197,  ..., -0.0921,  0.1819, -0.2634],
         [-0.0710, -0.3677, -0.0197,  ...,  0.2314,  0.1399, -0.2715],
         [ 0.1044, -0.0995,  0.0305,  ...,  0.0669,  0.0897, -0.2531]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=None, hidden_states=None, attentions=None)
Ouput embedding size: torch.Size([1, 197, 768])
Output embedding size: torch.Size([1, 20])


# Fine Tuning ViT Impelementation

In [5]:
#Pretrained model checkpoint
model_checkpoint = 'google/vit-base-patch16-224-in21k'

In [6]:
class ImageDataset(torch.utils.data.Dataset):

  def __init__(self, input_data):

      self.input_data = input_data
      self.transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((224, 224), antialias=True),
        transforms.Normalize(mean=[0.5, 0.5, 0.5],
                             std=[0.5, 0.5, 0.5])
        ])

  def __len__(self):
      return len(self.input_data)

  def get_images(self, idx):
      return self.transform(self.input_data[idx]['image'])

  def get_labels(self, idx):
      return self.input_data[idx]['label']

  def __getitem__(self, idx):
      # Get input data in a batch
      train_images = self.get_images(idx)
      train_labels = self.get_labels(idx)

      return train_images, train_labels

In [7]:
def train_model(model, train_dataset, epochs, learning_rate, batch_size):

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    model = model.to(device)

    train_loader = DataLoader(ImageDataset(train_dataset), batch_size=batch_size, shuffle=True, num_workers=1)
    criterion = nn.CrossEntropyLoss().to(device)
    optimizer = Adam(model.parameters(), lr=learning_rate)

    for epoch in range(epochs):
        model.train()
        total_loss, total_acc = 0, 0

        for images, labels in tqdm(train_loader):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            total_acc += (outputs.argmax(dim=1) == labels).sum().item()

        avg_loss = total_loss / len(train_loader.dataset)
        avg_acc = total_acc / len(train_loader.dataset)
        print(f'Epoch {epoch + 1}/{epochs} - Loss: {avg_loss:.4f}, Accuracy: {avg_acc:.4f}')

    return model

def test_model(model, test_dataset, batch_size):

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    model = model.to(device)

    test_loader = DataLoader(ImageDataset(test_dataset), batch_size=batch_size, shuffle=False, num_workers=1)

    model.eval()
    criterion = nn.CrossEntropyLoss().to(device)

    total_loss, total_acc = 0, 0
    with torch.no_grad():
        for images, labels in tqdm(test_loader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            total_acc += (outputs.argmax(dim=1) == labels).sum().item()

    avg_loss = total_loss / len(test_loader.dataset)
    avg_acc = total_acc / len(test_loader.dataset)
    print(f'Test - Loss: {avg_loss:.4f}, Accuracy: {avg_acc:.4f}')

    return avg_loss, avg_acc

In [8]:
import torch
import torch.nn as nn
from transformers import ViTModel, ViTConfig

class ViT(nn.Module):
    def __init__(self, config=ViTConfig(), num_labels=20,
                 model_checkpoint='google/vit-base-patch16-224-in21k',
                 encoded_weights=None, codec=None):
        super(ViT, self).__init__()
        self.vit = ViTModel.from_pretrained(model_checkpoint, add_pooling_layer=False)
        self.classifier = nn.Linear(config.hidden_size, num_labels)
        self.encoded_weights = encoded_weights
        self.codec = codec



In [9]:
# Hyperparameters
EPOCHS = 3
LEARNING_RATE = 1e-4
BATCH_SIZE = 32
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")



In [ ]:
model = ViT().to(device)
trained_model = train_model(model, dataset['train'], EPOCHS, LEARNING_RATE, BATCH_SIZE)

In [10]:
# load weights if you have it locally so don't have to fine tune again

trained_model =  ViT()
trained_model.load_state_dict(torch.load('original_model.pth'))

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTModel: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing ViTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>

In [11]:
test_loss, test_accuracy = test_model(trained_model, dataset['test'], BATCH_SIZE)
print(test_loss, test_accuracy)

100%|██████████| 30/30 [00:10<00:00,  2.78it/s]

Test - Loss: 0.0131, Accuracy: 0.8960
0.013065653626157456 0.8960084033613446


In [12]:
ViT()

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTModel: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing ViTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


ViT(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=768, out_features=3072,

# Round model weights

In [13]:
import copy


trained_model.eval()  # Set the model to evaluation mode
model.to('cpu')  # Move the model to CPU

rounded_model = copy.deepcopy(trained_model)
rounded_model.to('cpu')
precision = 2

def round_with_decimal_precision(tensor, decimal_precision=2):
    # Convert the PyTorch tensor to a NumPy array
    numpy_array = tensor.numpy()

    # Apply rounding with decimal precision using NumPy
    rounded_numpy_array = np.round(numpy_array, decimals=decimal_precision)

    # Convert the rounded NumPy array back to a PyTorch tensor
    rounded_tensor = torch.tensor(rounded_numpy_array, dtype=tensor.dtype)

    return rounded_tensor




for name, param in rounded_model.named_parameters():
    param.data = round_with_decimal_precision(param.data, decimal_precision=precision).to(torch.float16)
    print(param)
    print(param.dtype)


Streaming output truncated to the last 5000 lines.
Parameter containing:
tensor([ 0.1300,  0.1801, -0.1000,  0.0300, -0.0100, -0.0100,  0.0200, -0.1100,
         0.0200, -0.1400, -0.0100,  0.0000, -0.0500,  0.0200, -0.0400, -0.1000,
         0.0800,  0.0400,  0.0900,  0.0700, -0.0400,  0.0800, -0.0000, -0.0900,
        -0.0500,  0.0400, -0.0300,  0.0100,  0.0400, -0.1100,  0.0700, -0.0100,
        -0.1000, -0.1100, -0.0700,  0.0800,  0.0900,  0.1100, -0.0400, -0.0500,
         0.0400,  0.0200,  0.0400,  0.0900,  0.0800,  0.0200, -0.1500, -0.1200,
         0.0200,  0.0300,  0.0500,  0.2600, -0.0500, -0.0100,  0.0300,  0.0100,
        -0.0700, -0.0100, -0.0500,  0.0400,  0.0600,  0.0500,  0.0200,  0.1700,
         0.0700, -0.0200, -0.0500, -0.0200,  0.0200,  0.1000, -0.0600,  0.0000,
        -0.0300,  0.0300,  0.0800, -0.0400,  0.0100, -0.1000, -0.0600, -0.0500,
        -0.0400,  0.0500,  0.0100,  0.0500,  0.0000,  0.1000, -0.2800,  0.0200,
        -0.0100, -0.0500,  0.0900, -0.1000, -0.

In [14]:
print("Original Weights:")
for param in trained_model.parameters():
    print(param.data)
    print(param.dtype)

Streaming output truncated to the last 5000 lines.
        -3.3153e-02, -9.0043e-03,  3.3234e-02,  7.3817e-02,  4.2935e-02,
        -2.8925e-02,  5.6839e-02, -2.7244e-02,  6.5364e-02, -6.1048e-02,
         5.6452e-02, -7.1607e-04,  1.1128e-02,  6.7977e-02, -4.9994e-02,
         4.0885e-02, -3.9867e-02, -2.7253e-02, -3.6597e-02, -2.3783e-02,
         8.9503e-02,  3.5000e-02,  3.4659e-02, -7.7352e-02, -3.1295e-05,
         1.5729e-02,  1.3709e-02,  2.8197e-03,  9.2610e-02,  5.9007e-02,
        -3.2807e-02, -5.9810e-02, -3.8020e-02, -2.7054e-02,  1.8600e-02,
         2.6674e-02, -7.8853e-02,  2.7106e-02, -3.5139e-02,  2.7321e-02,
         4.3212e-02, -5.1611e-02, -5.0318e-02, -5.9760e-02,  1.9869e-02,
         2.3637e-02,  9.1467e-03,  6.5357e-02,  3.1776e-02,  7.3085e-02,
        -2.7036e-02, -4.6834e-02, -3.5431e-02, -4.2530e-03,  1.2224e-02,
        -1.4502e-02,  1.0337e-01, -5.1749e-02,  5.3031e-02, -7.5631e-03,
        -4.2693e-02, -4.0983e-02, -4.6262e-02, -1.3562e-03, -1.3429e-02,


In [15]:
def test_model_rounded(model, test_dataset, batch_size):

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    test_loader = DataLoader(ImageDataset(test_dataset), batch_size=batch_size, shuffle=False, num_workers=1)

    model.eval()
    model.to(device)

    criterion = nn.CrossEntropyLoss().to(device)

    total_loss, total_acc = 0, 0
    with torch.no_grad():
        for images, labels in tqdm(test_loader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            total_acc += (outputs.argmax(dim=1) == labels).sum().item()

    avg_loss = total_loss / len(test_loader.dataset)
    avg_acc = total_acc / len(test_loader.dataset)
    print(f'Test - Loss: {avg_loss:.4f}, Accuracy: {avg_acc:.4f}')

    return avg_loss, avg_acc

# Test the model
test_loss, test_accuracy = test_model_rounded(rounded_model, dataset['test'], BATCH_SIZE)
print(test_loss, test_accuracy)

100%|██████████| 30/30 [00:04<00:00,  6.95it/s]

Test - Loss: 0.0131, Accuracy: 0.8971
0.013088450712316176 0.8970588235294118


In [16]:
import torch
import os
model.to("cpu")
# Assuming 'model' is your original ViT model and 'quantized_model' is the quantized version

# Save the original model
torch.save(trained_model.state_dict(), 'original_model.pth')

# Save the quantized model
torch.save(rounded_model.state_dict(), 'rounded_model.pth')

# Function to get file size in MB
def file_size_in_mb(file_path):
    size_bytes = os.path.getsize(file_path)
    return size_bytes / (1024 * 1024)  # Convert bytes to megabytes

# Print the file sizes
original_size = file_size_in_mb('original_model.pth')
rounded_model_size = file_size_in_mb('rounded_model.pth')
print(f"Original Model Size: {original_size:.2f} MB")
print(f"Rounded Model Size: {rounded_model_size:.2f} MB")

Original Model Size: 327.43 MB
Rounded Model Size: 163.76 MB


# Huffman encoding

In [58]:
for  name, module in rounded_model.named_modules():
  print(module)

ViT(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=768, out_features=3072,

In [17]:
# Collect encoding statistics


device = torch.device("cpu")
rounded_model.to(device)


all_weights_flat = []


for name, param in rounded_model.named_parameters():
    layer_weights = param.data.cpu().numpy().flatten().tolist()
    all_weights_flat.extend(layer_weights)


frequencies = Counter(all_weights_flat)
codec = HuffmanCodec.from_frequencies(frequencies)

print(frequencies)

Counter({-0.0: 4801396, -0.01000213623046875: 4718380, 0.01000213623046875: 4713632, -0.0200042724609375: 4518966, 0.0200042724609375: 4512405, -0.029998779296875: 4232339, 0.029998779296875: 4218957, -0.040008544921875: 3886879, 0.040008544921875: 3880155, -0.04998779296875: 3508039, 0.04998779296875: 3505007, -0.05999755859375: 3125924, 0.05999755859375: 3114815, -0.07000732421875: 2742954, 0.07000732421875: 2737288, -0.08001708984375: 2375581, 0.08001708984375: 2370123, -0.09002685546875: 2028457, 0.09002685546875: 2028266, 0.0999755859375: 1715050, -0.0999755859375: 1715035, -0.1099853515625: 1437913, 0.1099853515625: 1436040, 0.1199951171875: 1192918, -0.1199951171875: 1189551, 0.1300048828125: 982196, -0.1300048828125: 977143, 0.1400146484375: 801577, -0.1400146484375: 797750, 0.1500244140625: 650085, -0.1500244140625: 644963, 0.1600341796875: 522809, -0.1600341796875: 518854, 0.1700439453125: 420057, -0.1700439453125: 415979, 0.1800537109375: 334575, -0.1800537109375: 332156, 0.

In [18]:
huffman_model = copy.deepcopy(rounded_model)
encoded_weights_dict = {}



for name, param in huffman_model.named_parameters():
    layer_weights = param.data.flatten().tolist()
    shape = param.shape
    encoded_weights = codec.encode(layer_weights)
    encoded_weights_dict[name] = (encoded_weights, shape)
    decoded_weights = codec.decode(encoded_weights)
    assert layer_weights == decoded_weights, "Mismatch in encoding/decoding!"

torch.save(encoded_weights_dict, 'encoded_model_weights.pth')

In [19]:
()
huffman_size = file_size_in_mb('encoded_model_weights.pth')
print(f"Huffman Model Size: {huffman_size:.2f} MB")

Huffman Model Size: 80.14 MB


# Test model

In [20]:
def test_model_huffman(model, test_dataset, batch_size, codec, encoded_weights_file):

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    test_loader = DataLoader(ImageDataset(test_dataset), batch_size=batch_size, shuffle=False, num_workers=1)

    model = model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)

    total_loss, total_acc = 0, 0
    with torch.no_grad():
        for images, labels in tqdm(test_loader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            total_acc += (outputs.argmax(dim=1) == labels).sum().item()

    avg_loss = total_loss / len(test_loader.dataset)
    avg_acc = total_acc / len(test_loader.dataset)
    print(f'Test - Loss: {avg_loss:.4f}, Accuracy: {avg_acc:.4f}')

    return avg_loss, avg_acc, model.state_dict()

## Test entire decoded Huffman model

In [21]:
encoded_weights_dict = torch.load('encoded_model_weights.pth')
huffman_test_model = ViT()
huffman_test_model.to(device)

for name, param in huffman_test_model.named_parameters():
    if name in encoded_weights_dict:
        decoded_weights = codec.decode(encoded_weights_dict[name][0])
        decoded_weights_tensor = torch.tensor(decoded_weights, device=device).view_as(param.data)
        param.data.copy_(decoded_weights_tensor)
    else:
        print(f"Weight for layer {name} not found in encoded weights.")

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTModel: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing ViTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [22]:
huffman_model.eval()
test_loss, test_accuracy, state_dic = test_model_huffman(huffman_model, dataset['test'], BATCH_SIZE, codec, 'encoded_model_weights.pth')
print(test_loss, test_accuracy)

100%|██████████| 30/30 [00:04<00:00,  6.94it/s]

Test - Loss: 0.0131, Accuracy: 0.8971
0.013088450712316176 0.8970588235294118


## Test dynamically decoded huffman model

In [26]:
def test_dynamic_huffman_model(model, test_dataset, batch_size, codec, encoded_weights_file):

    use_cuda = torch.cuda.is_available()
    device = torch.device( "cpu")
    test_loader = DataLoader(ImageDataset(test_dataset), batch_size=batch_size, shuffle=False, num_workers=1)

    model = model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)

    total_loss, total_acc = 0, 0
    with torch.no_grad():
        for images, labels in tqdm(test_loader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            total_acc += (outputs.argmax(dim=1) == labels).sum().item()

    avg_loss = total_loss / len(test_loader.dataset)
    avg_acc = total_acc / len(test_loader.dataset)
    print(f'Test - Loss: {avg_loss:.4f}, Accuracy: {avg_acc:.4f}')

    return avg_loss, avg_acc, model.state_dict()

In [56]:
encoded_weights_dict = torch.load('encoded_model_weights.pth')


# class DynamicHuffmanViT(nn.Module):
#     def __init__(self, codec, encoded_weights_dict):
#         super(DynamicHuffmanViT, self).__init__()
#         self.codec = codec
#         self.encoded_weights_dict = encoded_weights_dict

#     def decode_weights(self, encoded_weights):
#
#         decoded_weights = self.codec.decode(encoded_weights)
#         return decoded_weights

#     def encode_weights(self, decoded_weights):
#
#         encoded_weights = self.codec.encode(decoded_weights)
#         return encoded_weights

#     def get_bias_layer(self, layer_name)

#     def forward(self, x):
#         for layer_name, (encoded_weights, original_shape) in self.encoded_weights_dict.items():

#             if name.endswith('.bias'):
#               continue
#             # Decode weights dynamically
#             decoded_weights = self.decode_weights(encoded_weights)

#             # Separate weights and biases
#             weight_decoded, bias_decoded = decoded_weights

#             # Reshape weights and biases
#             reshaped_weight_array = np.array(weight_decoded).reshape(original_shape)
#             reshaped_weight = torch.tensor(reshaped_weight_array)

#             reshaped_bias_array = np.array(bias_decoded).reshape(original_shape[0]) if bias_decoded is not None else None
#             reshaped_bias = torch.tensor(reshaped_bias_array) if reshaped_bias_array is not None else None

#             # Forward pass through the current layer
#             x = nn.functional.linear(x, weight=reshaped_weight, bias=reshaped_bias)

#             # Encode the weights and biases back after computation
#             encoded_weight = self.encode_weights(reshaped_weight.view(-1).numpy())
#             encoded_bias = self.encode_weights(reshaped_bias.view(-1).numpy()) if reshaped_bias is not None else None

#             # Update the encoded weights and original shape in the dictionary
#             self.encoded_weights_dict[layer_name] = ([encoded_weight, encoded_bias], original_shape)

#         return x



class DynamicHuffmanViT(nn.Module):
    def __init__(self, codec, encoded_weights_dict):
        super(DynamicHuffmanViT, self).__init__()
        self.codec = codec
        self.encoded_weights_dict = encoded_weights_dict

    def decode_params(self, encoded_layer, original_shape):
        decoded_params = self.codec.decode(encoded_layer)
        reshaped_params_array = np.array(decoded_params).reshape(original_shape)
        reshaped_params_tensor = torch.tensor(reshaped_params_array)
        return reshaped_params_tensor

    def forward(self, x):
        for layer_name, (encoded_layer, original_shape) in self.encoded_weights_dict.items():

            print(layer_name)

            if layer_name.endswith('.bias'):
                continue  # skip bias entry as it is fetched with weight

            # Decode weights dynamically
            decoded_weight = self.decode_params(encoded_layer, original_shape)

            if layer_name.endswith('.weight'):
                # Handle bias layer separately
                bias_layer_name = layer_name.replace(".weight", ".bias")
                bias_layer_encoded_layer, bias_layer_original_shape = self.encoded_weights_dict[bias_layer_name]
                decoded_bias = self.decode_params(bias_layer_encoded_layer, bias_layer_original_shape)


                x = nn.functional.linear(x, weight=decoded_weight, bias=decoded_bias)
            elif 'embedding' in layer_name:
                # embedding layers
                x = x + decoded_weight
            else:
                x = nn.functional.linear(x, weight=decoded_weight)

        return x



dynamic_model = ViTIncrementalDecode(codec, encoded_weights_dict)



embeddings
embeddings.patch_embeddings
embeddings.patch_embeddings.projection
embeddings.dropout
encoder
encoder.layer
encoder.layer.0
encoder.layer.0.attention
encoder.layer.0.attention.attention
encoder.layer.0.attention.attention.query
encoder.layer.0.attention.attention.key
encoder.layer.0.attention.attention.value
encoder.layer.0.attention.attention.dropout
encoder.layer.0.attention.output
encoder.layer.0.attention.output.dense
encoder.layer.0.attention.output.dropout
encoder.layer.0.intermediate
encoder.layer.0.intermediate.dense
encoder.layer.0.intermediate.intermediate_act_fn
encoder.layer.0.output
encoder.layer.0.output.dense
encoder.layer.0.output.dropout
encoder.layer.0.layernorm_before
encoder.layer.0.layernorm_after
encoder.layer.1
encoder.layer.1.attention
encoder.layer.1.attention.attention
encoder.layer.1.attention.attention.query
encoder.layer.1.attention.attention.key
encoder.layer.1.attention.attention.value
encoder.layer.1.attention.attention.dropout
encoder.layer.

In [28]:
dynamic_model.eval()
test_loss, test_accuracy, state_dic = test_dynamic_huffman_model(dynamic_model, dataset['test'], BATCH_SIZE, codec, 'encoded_model_weights.pth')
print(test_loss, test_accuracy)

  0%|          | 0/30 [00:00<?, ?it/s]

embeddings


  0%|          | 0/30 [00:06<?, ?it/s]


TypeError: ignored